In [1]:
import numpy as np
import tensorflow as tf
from scipy import interpolate
from model import Model, default_opt
from subpixel import SubPixel1D, SubPixel1D_v2
from standard import conv1d, deconv1d

In [2]:
# shape = (num of example, sampling-ratio, num of channel)
X = tf.placeholder(tf.float32, shape=(100, 2048, 1), name='X')
Y = tf.placeholder(tf.float32, shape=(100, 8192, 1), name='Y')
downsampled_l = []
di = 0

In [3]:
def reshape1Dto2D(X):
    n_batch, n_width, n_chan = X.get_shape()
    X = tf.reshape(X,[n_batch, 1, n_width, n_chan])
    return X

def reshape2Dto1D(X):
    # reshape 2D -> 1D
    n_batch, _, n_width, n_chan = X.get_shape()
    X = tf.reshape(X,[n_batch, n_width, n_chan])
    return X

def downsample_layer(x , nf, ks, B=False):
    x = tf.layers.conv1d(
        x,
        filters = nf,
        kernel_size = ks,
        strides=1,
        padding='same',
        data_format='channels_last',
        dilation_rate=1,
        activation=None,
        use_bias=True,
        kernel_initializer=None,
        bias_initializer=tf.zeros_initializer(),
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        trainable=True,
        name=None,
        reuse=None
    )
    x = tf.layers.max_pooling1d(
        x,
        pool_size = 2,
        strides = 2,
        padding='same',
        data_format='channels_last',
        name=None
    )
    
    if B : x = tf.layers.dropout(x, rate=0.5)
        
    x = tf.nn.relu(x)
    return x

def upsample_layer(x, nf, ks):
    '''x = tf.layers.conv2d_transpose(
        x,
        filters = nf,
        kernel_size = [1,ks],
        strides=(1, 1),
        padding='same',
        data_format='channels_last',
        activation=None,
        use_bias=True,
        kernel_initializer=None,
        bias_initializer=tf.zeros_initializer(),
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        trainable=True,
        name=None,
        reuse=None
    )'''
    x = tf.layers.conv1d(
        x,
        filters = nf,
        kernel_size = ks,
        strides=1,
        padding='same',
        data_format='channels_last',
        dilation_rate=1,
        activation=None,
        use_bias=True,
        kernel_initializer=None,
        bias_initializer=tf.zeros_initializer(),
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        trainable=True,
        name=None,
        reuse=None
    )
    x = tf.layers.dropout(x, rate=0.5)
    x = tf.nn.relu(x)
    x = SubPixel1D(x,r=2)
    return x

In [4]:
n_filters = [48,  96,  128, 128]
n_filtersizes = [24, 12, 6,  3]

# downsampling layers
for l, nf, fs in zip(range(LD), n_filters, n_filtersizes):
    X = downsample_layer(X, nf, fs)
    downsampled_l.append(X)
    print(X)
    
# Bottle net layer
X = downsample_layer(X, n_filters[-1], n_filtersizes[-1], B=True)
print(X)

# Upsample layer
LU = reversed(range(LU))
n_filters = reversed(n_filters)
n_filtersizes = reversed(n_filtersizes)
downsampled_l = reversed(downsampled_l)

for l, nf, fs in zip( LU, (n_filters), (n_filtersizes)):
    #X = reshape1Dto2D(X)
    X = upsample_layer(X, nf*2, fs)
    #X = reshape2Dto1D(X)
    #if l < len(downsampled_l): 
    #    l_in = downsampled_l[l]
    #    X = tf.concat([X,l_in],axis=-1)
    print('concated >> ',X)
    
# Final layer and add input layer
X = upsample_layer(X,nf=2,ks=9)
print(X)
print(oX)
X = tf.add(X,oX)
print(X)


Tensor("X:0", shape=(100, 2048, 1), dtype=float32)
D-Block-00: Tensor("Relu:0", shape=(100, 1024, 12), dtype=float32)
D-Block-01: Tensor("Relu_1:0", shape=(100, 512, 24), dtype=float32)
D-Block-02: Tensor("Relu_2:0", shape=(100, 256, 48), dtype=float32)
D-Block-03: Tensor("Relu_3:0", shape=(100, 128, 48), dtype=float32)
Bottle-Neck: Tensor("Relu_4:0", shape=(100, 64, 48), dtype=float32)
U-Block-03: Tensor("concat:0", shape=(100, 128, 96), dtype=float32)
U-Block-02: Tensor("concat_1:0", shape=(100, 256, 96), dtype=float32)
U-Block-01: Tensor("concat_2:0", shape=(100, 512, 48), dtype=float32)
U-Block-00: Tensor("concat_3:0", shape=(100, 1024, 24), dtype=float32)
U-Block-Fin1: Tensor("subpixel_4/transpose_1:0", shape=(100, 2048, 12), dtype=float32)
U-Block-Fin2: Tensor("subpixel_5/transpose_1:0", shape=(100, 4096, 12), dtype=float32)
Final: Tensor("subpixel_6/transpose_1:0", shape=(100, 8192, 1), dtype=float32)
X4: Tensor("subpixel_8/transpose_1:0", shape=(100, 8192, 1), dtype=float32)
Ge

In [5]:


'''
# Model config
#n_filters = [128, 256, 512,512]
nf = [12,  24,  48, 48, 48]
#n_filtersizes = [65, 33, 17,  9]
fs = [10, 7, 3, 3, 3]

print(X)

# D-level-00
dx0 = downsample_layer(X, nf[0], fs[0])
print('D-Block-00:',dx0)

# D-level-01
dx1 = downsample_layer(dx0, nf[1], fs[1])
print('D-Block-01:',dx1)

# D-level-02
dx2 = downsample_layer(dx1, nf[2], fs[2])
print('D-Block-02:',dx2)

# D-level-03
dx3 = downsample_layer(dx2, nf[3], fs[3])
print('D-Block-03:',dx3)

# Bottle-Neck
btn = downsample_layer(dx3, nf[4], fs[4])
print('Bottle-Neck:',btn)

# U-level-03 
ux3 = upsample_layer(btn, nf[3] * 2, fs[3])
ucx3 = tf.concat([ux3, dx3], axis=-1)
print('U-Block-03:',ucx3)

# U-level-02
ux2 = upsample_layer(ucx3, nf[2] * 2, fs[2])
ucx2 = tf.concat([ux2, dx2], axis=-1)
print('U-Block-02:',ucx2)

# U-level-01
ux1 = upsample_layer(ucx2, nf[1] * 2, fs[1])
ucx1 = tf.concat([ux1, dx1], axis=-1)
print('U-Block-01:',ucx1)

# U-level-00
ux0 = upsample_layer(ucx1, nf[0] * 2, fs[0])
ucx0 = tf.concat([ux0, dx0], axis=-1)
print('U-Block-00:',ucx0)

# U x 2
ux = upsample_layer(ucx0, nf[0] * 2, fs[0])
print('U-Block-Fin1:',ux)

# U x 2
ux = upsample_layer(ux, nf[0] * 2, fs[0])
print('U-Block-Fin2:',ux)

# Fin
G = upsample_layer(ux, 2, 9)
print('Final:',G)

# X2?
X4 = upsample_layer(X, 2, 9)
X4 = upsample_layer(X4, 2, 9)
print('X4:',X4)

# Generator
G = tf.add(G,X4)
print('Generator:',G)
'''

"\n# downsampling layers\nfor l, nf, fs in zip(range(LD), n_filters, n_filtersizes):\n    X = downsample_layer(X, nf, fs)\n    downsampled_l.append(X)\n    print(X)\n    \n# Bottle net layer\nX = downsample_layer(X, n_filters[-1], n_filtersizes[-1], B=True)\nprint(X)\n\n# Upsample layer\nLU = reversed(range(LU))\nn_filters = reversed(n_filters)\nn_filtersizes = reversed(n_filtersizes)\ndownsampled_l = reversed(downsampled_l)\n\nfor l, nf, fs in zip( LU, (n_filters), (n_filtersizes)):\n    #X = reshape1Dto2D(X)\n    X = upsample_layer(X, nf*2, fs)\n    #X = reshape2Dto1D(X)\n    #if l < len(downsampled_l): \n    #    l_in = downsampled_l[l]\n    #    X = tf.concat([X,l_in],axis=-1)\n    print('concated >> ',X)\n    \n# Final layer and add input layer\nX = upsample_layer(X,nf=2,ks=9)\nprint(X)\nprint(oX)\nX = tf.add(X,oX)\nprint(X)\n"